# Getting Started with TF SEAL

In this notebook, we will be working on how to do a few simple operations in [TF SEAL](https://github.com/tf-encrypted/tf-seal), a library that bridges between the Microsoft SEAL homomorphic encryption library and TensorFlow in Python. Before going through the rest of this notebook, familiarize yourself with the [Microsoft SEAL library](https://github.com/Microsoft/SEAL) in either C++ or .NET by working through their provided examples. 

## Setting up TF SEAL

For the rest of process, we will be using Ubuntu 20.04, and a Python 3.7 environment in Anaconda, as suggested by the developers of TF SEAL. If you do not have Anaconda on your device, go to the [Anaconda distribution page](https://www.anaconda.com/distribution/) and install the latest version for Linux for Python 3.7. Once you have Anaconda ready to go, we will create a conda environment, which I call `customtf`, and then activate it. The command to do this in the terminal is

```
conda create -n customtf python=3.7 -y
source activate customtf
```
Then, since the SEAL and TensorFlow libraries were compiled using different C++ versions, we get a patched version of the TensorFlow library compatible with SEAL, by typing

```
wget https://storage.googleapis.com/tf-pips/tf-c++17-support/tf_nightly-1.14.0-cp37-cp37m-linux_x86_64.whl
pip install tf_nightly-1.14.0-cp37-cp37m-linux_x86_64.whl
```

in the terminal. Make sure to have `pip` installed before doing this step. We can then finally use `pip` again to get the TF SEAL library:

```
pip install tf-seal
```


## Basic Operations in TF SEAL

We will demonstrate how to do the two basic homomorphic operations using TF SEAL, addition and multiplication. By generating random numbers, and doing both operations on encrypted and plain text values, then decrypting the result, we will verify that the operation is homomorphic. To begin with, we import the necessary libraries:

In [36]:
import numpy as np
import tensorflow as tf
import tf_seal as tfs

Since TF SEAL is to be used with TensorFlow, and not simply as a generic homomorphic encryption library, we require NumPy to generate matrices (or rather tensors) to do operations on. We then generate the public and secret keys necessary for encrypting and decrypting the data 

In [37]:
public_keys, secret_key = tfs.seal_key_gen(gen_relin=True, gen_galois=True)

We set the flags to generate the relinearize and galois keys as true, as both of those public keys are also necessary for efficient computations. Once the keys have been generated, the two random values to be added can be generated and encrypted

### Addition

In [56]:
a_plain = np.random.normal(size=(1, 2)).astype(np.float32)
print("a_plain =",a_plain)
a = tfs.constant(a_plain, secret_key, public_keys)

a_plain = [[0.33529496 0.04337829]]


`a_plain` will be used for plaintext computations to verify the encrypted computations on `a`. By printing the values of `a_plain` here, we can also manually verify the computations later on. Since TF SEAL is meant to be used with tensors in TensorFlow, rather than for generic operations on scalars, we can manipulate matrix multiplication to allow us to sum scalars.

$$
\begin{align*}
    a_{plain} 
    &= \left(
    \begin{array}{c c}
        a_{11} & a_{12}\\
    \end{array}
    \right)\\
    b 
    &= \left(
    \begin{array}{c}
        1\\
        1\\
    \end{array}
    \right)\\
    a_{plain} b
    &= \left(
    \begin{array}{c}
        a_{11}+a_{12}\\
    \end{array}
    \right)
\end{align*}
$$

From this, we can see that multiplying `a_plain` by a 2x1 matrix of 1s, we can arrive at a 1x1 matrix containing the sum of the two values, allowing us to compute the sum using the functions available in TF SEAL. Let us define a variable for `b`

In [57]:
b  = np.ones((2, 1)).astype(np.float32)

Now, we simply make two more variables holding the multiplied values

In [58]:
c = tfs.matmul(a, b.transpose())
d = tf.matmul(a_plain, b)

Lastly, we run a TensorFlow session to execute the operations, and print the outputs

In [60]:
with tf.compat.v1.Session() as sess:
    print("SEAL: ", sess.run(c))
    print("Plain:", sess.run(d))

SEAL:  [[0.37867507]]
Plain: [[0.37867326]]


Even with the approximate operations from homomorphic encryption, the two values are nearly identical! The summation operation can be verified as very nearly homomorphic. Although the values are not exactly the same, it is important to keep in mind that this will eventually be used for machine learning. The data used will also be noisy, and the operations would not result in exact values anyways, so minor errors are acceptable. 

### Multiplication

We can now move on to verify multiplication. Using similar steps as above, we will define 2 random 1x1 variables in plaintext, and encrypt them. Then, we will multiply the plaintext and encrypted values and compare.

In [61]:
a_plain = np.random.normal(size=(1,1)).astype(np.float32)
b_plain = np.random.normal(size=(1,1)).astype(np.float32)

print("a_plain =", a_plain)
print("b_plain =", b_plain)

a = tfs.constant(a_plain, secret_key, public_keys)
b = tfs.constant(b_plain, secret_key, public_keys)

c = tfs.matmul(a, b)
d = tf.matmul(a_plain, b_plain)

with tf.compat.v1.Session() as sess:
    print("\nSEAL: ", sess.run(c))
    print("Plain:", sess.run(d))

a_plain = [[-0.30829367]]
b_plain = [[-0.22017]]

SEAL:  [[0.06787702]]
Plain: [[0.06787702]]


Once again, the approximate operations from homomorphic encryption is nearly identical to multiplication on plaintext values. With these findings, we can move on to test a practical example where homomorphic encryption can be used.